# The Grid module

The grid module handles mesh creation/integration based on given boundaries. 


- Requires:

data folder. See [README](README.md) in this folder.

In [ ]:
#to use the full width of the browser window
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
# Not neccessarily needed. Just to check version. Must be > 0.4.1
import pyposeidon
pyposeidon.__version__

In [ ]:
import pyposeidon.grid as pg

### Graphics

In [ ]:
import pyposeidon.utils.pplot

In [ ]:
import pyposeidon.utils.hplot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
#optional
#pyposeidon.utils.pplot.__init__(dark_background=True) # set plt style for pplot graphics below

In [ ]:
#optional
#plt.style.use('dark_background')

In [ ]:
%matplotlib widget

In [ ]:
#Create a folder to save the output
import os
if not os.path.exists('test'):
            os.makedirs('test')

- In order to have within the Notebook the executable binaries installed in the conda env of the Kernel we add the corresponding path to the env PATH. This is not needed when we are running from python.

In [ ]:
cpath = pyposeidon.__path__[0].split('/lib/')[0] # get the current kernel path

os.environ['PATH'] += os.pathsep + cpath + '/bin' # add to PATH

## Setup

The minimum information required is the geometry's extent. In the most simple case that is a lat/lon box that defines the area of interest. Without loss of generality we select below Iceland as a test case. Feel free to modify the coordinates. 

In addition, the coastlines need to be provided as internal boundaries.

Altrenatively, the boundary can be given as a shapefile or GeoDataFrame.

In [ ]:
#define in a dictionary the properties of the model..
geometry={'lon_min':-25., # lat/lon window
     'lon_max':-9.,
     'lat_min':56.,
     'lat_max':74.,
    }

In [ ]:
# use cartopy to get coastlines
import cartopy.feature as cf

cr='i'

coast = cf.NaturalEarthFeature(
    category='physical',
    name='land',
    scale='{}m'.format({'l':110, 'i':50, 'h':10}[cr]))

In [ ]:
import geopandas as gp
ne_i = gp.GeoDataFrame(geometry = [x for x in coast.geometries()])

We need to provide the type of element that we want (for now 'r2d' or 'tri2d' ) and the engine for grid generation ('jigsaw' or 'gmsh').

*Note*: Currently 'jigsaw' remains the default.

In [ ]:
mesh = pg.grid(type='tri2d', geometry=geometry, coastlines = ne_i, grid_generator = 'jigsaw')

In [ ]:
mesh.Dataset

**TIP** : You can change the grid generator above with  grid_generator = 'gmsh'.

## Visualize

- with Matplolib

In [ ]:
mesh.Dataset.pplot.grid(lw=.5, markersize=.2)

One can overlay the coastlines on the graph with 

In [ ]:
aa=plt.gca()
ne_i.boundary.plot(ax=aa,color='orange')

- with Holoviews

In [ ]:
mesh.Dataset.hplot.grid()#tiles=True) 

## Control mesh size

Better control on the mesh size can be obtained by providing a background control function usually in the form of a scaled DEM. One way to achieve this is to give as argument a dem file, like : 

In [ ]:
mesh = pg.grid(type='tri2d', geometry=geometry, coastlines = ne_i, grid_generator = 'jigsaw', 
               bgmesh='auto', dem_source='data/dem.nc', resolution_min=0.01, resolution_max=.5)

In [ ]:
mesh.Dataset.pplot.grid(lw=.5, markersize=.2)

**TIP** : Change the grid generator above with  grid_generator = 'gmsh' to see how gmsh would do it.

## Output

In [ ]:
mesh.to_file('./test/hgrid.gr3')

## Read a given grid

In [ ]:
m = pg.grid(type='tri2d',grid_file='./test/hgrid.gr3')

In [ ]:
m.Dataset

### Grid Validation

If you model is SCHISM, the grid can be validated against it as 

In [ ]:
m.validate(rpath='./test/val/')

### Grid Verification

The grid is created with a specific boundary defined, usually, as a coastline. In order to check whether that coastline is crossed by the generated grid we can use :

In [ ]:
m.verify(coastlines=ne_i)

*Note* : This is still under development and there is no distiction between 'intersection' and 'touch' for the grid. That is, the nodes could very well be **on** the boundary as they should. 